In [1]:
import math
"""
Stores the basic instance of a stock option 
"""
class StockOption(object):
    def __init__(
        self, S0, K, r=0.05, T=1, N=2, pu=0, pd=0, 
        div=0, sigma=0, is_put=False, is_am=False):
        """
        :S0: initial stock price
        :K: strike price
        :r: risk-free interest rate
        :T: time to maturity
        :N: number of time steps
        :pu: probability at up state
        :pd: probability at down state
        :div: Dividend yield
        :param is_put: True for a put option,
                False for a call option
        :param is_am: True for an American option,
                False for a European option
        """
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N)
        self.STs = [] # Declare the stock prices tree

        """ Optional parameters used by derived classes """
        self.pu, self.pd = pu, pd
        self.div = div
        self.sigma = sigma
        self.is_call = not is_put
        self.is_european = not is_am

    @property
    #the single time step in years
    def dt(self):
        return self.T/float(self.N)
    
    @property
    #the discount factor
    def df(self):
        return math.exp(-(self.r-self.div)*self.dt)  



In [4]:
"""
Binomial option pricing is a type of iterative procedure that allows
for the specification of nodes during a time span between the option's
valuation and expiration date
"""
import math
import numpy as np
from decimal import Decimal

class BinomialEuropeanOption(StockOption):

    def setup_parameters(self):
        self.M = self.N+1 #Number of terminal nodes of tree
        self.u = 1+self.pu #Expected value in the up state
        self.d = 1 - self.pd #Expected value in the down state
        self.qu = (math.exp(
            (self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def init_stock_price_tree(self):
        #Initialize terminal price nodes to zeros
        self.STs = np.zeros(self.M)

        #Calculate expected stock prices for each node
        for i in range(self.M):
            self.STs[i] = self.S0 * \
                (self.u**(self.N - i)) * (self.d**i)
    
    def init_payoffs_tree(self):

        """
        Returns the payoffs when the option expires at terminal 
        nodes
        """
        if self.is_call:
            return np.maximum(0, self.STs-self.K)
        else:
            return np.maximum(0, self.K-self.STs)   
    def traverse_tree(self, payoffs):
        """
        Starting from the time the option expires, traverse
        backwards and calculate discounted payoffs at each node
        """
        for i in range(self.N):
            payoffs = (payoffs[:-1]*self.qu + 
                        payoffs[1:]*self.qd)*self.df

        return payoffs

    def begin_tree_traversal(self):
        payoffs = self.init_payoffs_tree()
        return self.traverse_tree(payoffs)

    def price(self):
        """ Entry point of the pricing implementation """
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        
        # Option value converges to first node
        return payoffs[0]


In [5]:
eu_option = BinomialEuropeanOption(
    50, 52, r=0.05, T=2, N=2, pu=0.2, pd=0.2, is_put=True)
print('European put option price is:', eu_option.price())

European put option price is: 4.1926542806038585


In [8]:
import math
import numpy as np

"""
Price of a american option by a binomial tree
"""
class BinomialTreeOption(StockOption):
    
    def setup_parameters(self):
        self.u = 1+self.pu  # Expected value in the up state
        self.d = 1-self.pd  # Expected value in the down state
        self.qu = (math.exp(
            (self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def init_stock_price_tree(self):
        # Initialize a 2D tree at T=0
        self.STs = [np.array([self.S0])]

        # Simulate the possible stock prices path
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate(
                (prev_branches*self.u, 
                 [prev_branches[-1]*self.d]))
            self.STs.append(st) # Add nodes at each time step

    def init_payoffs_tree(self):
        if self.is_call:
            return np.maximum(0, self.STs[self.N]-self.K)
        else:
            return np.maximum(0, self.K-self.STs[self.N])

    def check_early_exercise(self, payoffs, node):
        if self.is_call:
            return np.maximum(payoffs, self.STs[node] - self.K)
        else:
            return np.maximum(payoffs, self.K - self.STs[node])

    def traverse_tree(self, payoffs):
        for i in reversed(range(self.N)):
            # The payoffs from NOT exercising the option
            payoffs = (payoffs[:-1]*self.qu + 
                       payoffs[1:]*self.qd)*self.df

            # Payoffs from exercising, for American options
            if not self.is_european:
                payoffs = self.check_early_exercise(payoffs,i)

        return payoffs

    def begin_tree_traversal(self):
        payoffs = self.init_payoffs_tree()
        return self.traverse_tree(payoffs)

    def price(self):
        """  The pricing implementation """
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        return payoffs[0]

am_option = BinomialTreeOption(50, 52, 
    r=0.05, T=2, N=2, pu=0.2, pd=0.2, is_put=True, is_am=True)

print("American put option price is:", am_option.price())

American put option price is: 5.089632474198373


In [9]:
"""
Cox-Ross_Rubinstein model takes into account the voltality, or the standard
deviation returns of a stock, into acount
"""

import math
class BinomialCRROption(BinomialTreeOption):
    def setup_parameters(self):
        self.u = math.exp(self.sigma * math.sqrt(self.dt))
        self.d = 1./self.u
        self.qu = (math.exp((self.r-self.div)*self.dt) - 
                   self.d)/(self.u-self.d)
        self.qd = 1-self.qu
eu_option = BinomialCRROption(
    50, 52, r=0.05, T=2, N=2, sigma=0.3, is_put=True)
print('European put:', eu_option.price())
am_option = BinomialCRROption(
    50, 52, r=0.05, T=2, N=2, 
    sigma=0.3, is_put=True, is_am=True)
print('American put option price is:', am_option.price())

European put: 6.245708445206436
American put option price is: 7.428401902704834


In [10]:
""" The Leisen Reimer Tree approximates to the Black-Scholes solution as the number 
of step increases. Additionally, the nodes do not recombine at every alternate step, 
an an inversion formula is used during tree traversal.
"""

import math
import numpy as np

class BinomialLROption(BinomialTreeOption):

    def setup_parameters(self):
        odd_N = self.N if (self.N%2 == 0) else (self.N+1)
        d1 = (math.log(self.S0/self.K) +
              ((self.r-self.div) +
               (self.sigma**2)/2.)*self.T)/\
            (self.sigma*math.sqrt(self.T))
        d2 = (math.log(self.S0/self.K) +
              ((self.r-self.div) -
               (self.sigma**2)/2.)*self.T)/\
            (self.sigma * math.sqrt(self.T))

        pbar = self.pp_2_inversion(d1, odd_N)
        self.p = self.pp_2_inversion(d2, odd_N)
        self.u = 1/self.df * pbar/self.p
        self.d = (1/self.df-self.p*self.u)/(1-self.p)
        self.qu = self.p
        self.qd = 1-self.p

    def pp_2_inversion(self, z, n):
        return .5 + math.copysign(1, z)*\
            math.sqrt(.25 - .25*
                math.exp(
                    -((z/(n+1./3.+.1/(n+1)))**2.)*(n+1./6.)
                )
            )

eu_option = BinomialLROption(
    50, 52, r=0.05, T=2, N=4, sigma=0.3, is_put=True)
print("European put:", eu_option.price())

am_option = BinomialLROption(50, 52, 
    r=0.05, T=2, N=4, sigma=0.3, is_put=True, is_am=True)

print('American put:', am_option.price())

European put: 5.878650106601964
American put: 6.763641952939979
